## Dataset Preparation and Redundancy Reduction

In this notebook, Pandas and SQL will be used to examine the dataset and identify redundant columns across multiple tables. The goal of this process will be to simplify the dataset structure while preserving all necessary relationships between tables.

The analysis will focus on columns that store the same information in different tables, such as vendor names appearing alongside vendor identifiers (`VendorNumber`, `VendorNo`) or product attributes like `Brand`, `Description`, and `Size` repeated across inventory, purchase, and sales tables. Where appropriate, these redundant columns will be removed and replaced by unique identifiers to maintain consistency.

After dropping these redundant columns, the overall dataset size will be reduced from **3.50 GB** to **1.57 GB**, representing a decrease of approximately **55.1%**, without any loss of information. This reduction will improve storage efficiency, reduce processing overhead, and result in a cleaner, more normalized dataset that is easier to query and analyze using SQL and Pandas.



## Project Overview
This cell explains the goal of the notebook: analyzing a MySQL database, identifying redundant columns across tables, and normalizing the schema to reduce storage size while preserving information.

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import pymysql
from sqlalchemy import text

engine = create_engine(
    "mysql+pymysql://root:@localhost:3306/MyProject",
    echo=False
)


## Import Libraries & Create Database Connection
This cell imports required Python libraries and creates a reusable SQLAlchemy engine connected to the MyProject MySQL database.

In [81]:

query = """
SELECT 
    table_schema AS database_name,
    ROUND(SUM(data_length + index_length) / 1024 / 1024, 2) AS size_mb
FROM information_schema.tables
WHERE table_schema = DATABASE()
GROUP BY table_schema;
"""

df = pd.read_sql(query, engine)
df


,database_name,size_mb
0,myproject,3504.16


## Check Total Database Size
This cell queries `information_schema.tables` to calculate the total database size (data + indexes) and loads the result into a Pandas DataFrame.

In [82]:
df = pd.read_sql("SHOW TABLES;", engine)
for table in df['Tables_in_myproject']:
    print(f'{table}',' -----> Count of records: ',pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", engine) ['count'].values[0])

begin_inventory  -----> Count of records:  206529
end_inventory  -----> Count of records:  224489
purchase_prices  -----> Count of records:  12261
purchases  -----> Count of records:  2372474
sales  -----> Count of records:  12825363
vendor_invoice  -----> Count of records:  5543


## Count Records in Each Table
This cell retrieves all table names and prints the total number of rows in each table to understand table scale.

In [83]:
for table in df['Tables_in_myproject']:
    print('\n\n','-'*50, f'{table}','-'*50,'\n\n')
    display(pd.read_sql(f"select * from {table} limit 3", engine))



 -------------------------------------------------- begin_inventory -------------------------------------------------- 




,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01




 -------------------------------------------------- end_inventory -------------------------------------------------- 




,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31




 -------------------------------------------------- purchase_prices -------------------------------------------------- 




...

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05




 -------------------------------------------------- sales -------------------------------------------------- 




,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,0.79,12546,JIM BEAM BRANDS COMPANY




 -------------------------------------------------- vendor_invoice -------------------------------------------------- 




,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61


## Preview Sample Data from Each Table
This cell displays the first three rows from each table to inspect schemas and identify redundant attributes.

In [84]:


query = """
SELECT brand, COUNT(*) AS count_occurrences
FROM purchase_prices
GROUP BY brand
HAVING COUNT(*) > 1;
"""

df = pd.read_sql(query, engine)
df.shape[0]


0

## Detect Duplicate Brands in purchase_prices
This cell groups the purchase_prices table by brand and identifies brands that appear more than once.

In [85]:

query = """
SELECT Description, PurchasePrice, COUNT(DISTINCT brand) AS distinct_brand_count
FROM purchase_prices
GROUP BY Description, PurchasePrice
HAVING COUNT(DISTINCT brand) > 1;
"""

df = pd.read_sql(query, engine)
df.shape[0]

0

## Check Brand Inconsistencies
This cell checks whether the same product description and price are associated with multiple brands.

In [89]:

query = """
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'purchase_prices'
INTERSECT
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'purchases';
"""

df = pd.read_sql(query, engine)
df

,column_name
0,Brand
1,Description
2,Size
3,PurchasePrice
4,VendorNumber
5,VendorName


## Find Shared Columns: purchase_prices vs purchases
This cell identifies overlapping column names between purchase_prices and purchases to detect redundancy.

In [96]:

query = """
ALTER TABLE purchases
DROP COLUMN IF EXISTS Description,
DROP COLUMN IF EXISTS Size,
DROP COLUMN IF EXISTS PurchasePrice,
DROP COLUMN IF EXISTS VendorNumber,
DROP COLUMN IF EXISTS VendorName;
"""

with engine.begin() as conn:  # auto-commit
    conn.execute(text(query))

## Drop Redundant Columns from purchases
This cell removes duplicated descriptive columns from the purchases table, keeping only IDs for normalization.

In [97]:

query = """
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'purchase_prices'
INTERSECT
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'end_inventory';
"""

df = pd.read_sql(query, engine)
df

,column_name
0,Brand
1,Description
2,Price
3,Size


## Find Shared Columns: purchase_prices vs end_inventory
This cell identifies overlapping columns between price and inventory tables.

In [98]:
query = """
ALTER TABLE end_inventory
DROP COLUMN IF EXISTS Description,
DROP COLUMN IF EXISTS Size,
DROP COLUMN IF EXISTS Price;
"""

with engine.begin() as conn:  # auto-commit
    conn.execute(text(query))

query = """
ALTER TABLE begin_inventory
DROP COLUMN IF EXISTS Description,
DROP COLUMN IF EXISTS Size,
DROP COLUMN IF EXISTS Price;
"""

with engine.begin() as conn:  # auto-commit
    conn.execute(text(query))

## Drop Redundant Columns from Inventory Tables
This cell removes redundant descriptive columns from begin_inventory and end_inventory tables.

In [99]:
query = """
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'purchase_prices'
INTERSECT
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'sales';
"""

df = pd.read_sql(query, engine)
df

,column_name
0,Brand
1,Description
2,Size
3,Volume
4,VendorName


## Find Shared Columns: purchase_prices vs sales
This cell detects duplicated descriptive columns between purchase_prices and sales tables.

In [100]:
query = """
ALTER TABLE sales
DROP COLUMN IF EXISTS Description,
DROP COLUMN IF EXISTS Size,
DROP COLUMN IF EXISTS VendorName,
DROP COLUMN IF EXISTS VendorNo,
DROP COLUMN IF EXISTS Volume;
"""

with engine.begin() as conn:  # auto-commit
    conn.execute(text(query))

## Drop Redundant Columns from sales
This cell removes repeated descriptive attributes from the sales table, leaving transactional data and keys only.

In [110]:
df = pd.read_sql("SHOW TABLES;", engine)

for table in df['Tables_in_myproject']:
    print('\n\n','-'*50, f'{table}','-'*50,'\n\n')
    display(pd.read_sql(f"select * from {table} limit 3", engine))



 -------------------------------------------------- begin_inventory -------------------------------------------------- 




,InventoryId,Store,City,Brand,onHand,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,8,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,7,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,6,2024-01-01




 -------------------------------------------------- end_inventory -------------------------------------------------- 




,InventoryId,Store,City,Brand,onHand,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,11,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,7,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,7,2024-12-31




 -------------------------------------------------- purchase_prices -------------------------------------------------- 




...

,InventoryId,Store,Brand,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,Quantity
0,69_MOUNTMEND_8412,69,8412,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,6
1,30_CULCHETH_5255,30,5255,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,4
2,34_PITMERDEN_5215,34,5215,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,5




 -------------------------------------------------- sales -------------------------------------------------- 




,InventoryId,Brand,SalesQuantity,SalesDate,ExciseTax
0,1_HARDERSFIELD_1004,1004,1,2024-01-01,0.79
1,1_HARDERSFIELD_1004,1004,2,2024-01-02,1.57
2,1_HARDERSFIELD_1004,1004,1,2024-01-03,0.79




 -------------------------------------------------- vendor_invoice -------------------------------------------------- 




,VendorNumber,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight
0,105,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47
1,4466,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57
2,388,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61


## Recheck Table Samples After Cleanup
This cell previews the tables again to verify that schema cleanup was successful.

In [111]:
query = """
SELECT 
    table_schema AS database_name,
    ROUND(SUM(data_length + index_length) / 1024 / 1024, 2) AS size_mb
FROM information_schema.tables
WHERE table_schema = DATABASE()
GROUP BY table_schema;
"""

df = pd.read_sql(query, engine)
df

,database_name,size_mb
0,myproject,1572.97


## Recalculate Database Size
This cell recalculates the total database size to confirm storage reduction after normalization.